In [6]:
import pyodbc
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

def get_informix_connection():
    driver = 'informix_Producao_BCARGA' 
    user = 'classprod'
    password = 'yY5eQ4vT'
    database = 'central'
    server = 'bd1hom'
    
    connection_string = f'DSN={driver};UID={user};PWD={password};DATABASE={database};SERVER={server}'
    connection = pyodbc.connect(connection_string)
    return connection

def query_informix(date_range, cnpjs):
    cnpj_str = ', '.join(f"'{cnpj}'" for cnpj in cnpjs)
    query = f"""
    SELECT nfe.sqnfeletronica, 
           hin.nrdocumento AS cnpj, 
           itemnfe.dsproduto, 
           itemnfe.vlproduto, 
           itemnfe.qtproduto, 
           itemnfe.vlbasecalcicms, 
           itemnfe.sgunidmedcom
    FROM tbfis_nfeletronica nfe
    JOIN tbfis_itemnfe itemnfe ON nfe.sqnfeletronica = itemnfe.sqnfeletronica
    JOIN tbfis_hinfeletron hin ON nfe.sqemitente = hin.sqhinfeletron
    WHERE hin.nrdocumento IN ({cnpj_str})
      AND nfe.dhemissao BETWEEN '{date_range[0]}' AND '{date_range[1]}'
    """
    connection = get_informix_connection()
    
    try:
        result_df = pd.read_sql(query, connection)
        return result_df
    except pyodbc.Error as e:
        print("Error: ", e)
        cursor = connection.cursor()
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        print("Columns in the query: ", columns)
        cursor.close()
    finally:
        connection.close()

def fetch_data_parallel(date_ranges, cnpjs):
    results = []
    with ThreadPoolExecutor(max_workers=40) as executor:
        future_to_date_range = {executor.submit(query_informix, date_range, cnpjs): date_range for date_range in date_ranges}
        for future in as_completed(future_to_date_range):
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
            except Exception as exc:
                print(f"Generated an exception: {exc}")
    
    if results:
        final_df = pd.concat(results, ignore_index=True)
        return final_df
    return pd.DataFrame()

if __name__ == "__main__":
    date_ranges = [
        ('2024-04-10 00:00:00', '2024-04-10 23:59:59')
    ]
    df_contribuintes = pd.read_excel(r'C:\Users\jamil\OneDrive\Área de Trabalho\Infomix-Sefaz\arquivos\CNPJS_07_06_24.xlsx')
    cnpj_contribuintes = df_contribuintes['CNPJ'].astype(str).tolist()
    cnpj_contribuintes = [re.sub(r'\D', '', cnpj).zfill(14) for cnpj in cnpj_contribuintes]
    lote = cnpj_contribuintes
    result_df = fetch_data_parallel(date_ranges, lote)
    if not result_df.empty:
        print(result_df)



c:\Users\jamil\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


       sqnfeletronica            cnpj  \
0           598301298  00289351000164   
1           598301298  00289351000164   
2           598320375  00388857000120   
3           598320375  00388857000120   
4           598363453  00388857000120   
...               ...             ...   
85830       598378615  93209765033120   
85831       598378615  93209765033120   
85832       598378615  93209765033120   
85833       598378612  93209765033120   
85834       598317062  93209765033120   

                                               dsproduto  vlproduto  \
0      COND SPLIT GREE G-TOP INVERTER CONNECTION - GW...    5070.00   
1      EVAP SPLIT GREE G-TOP INVERTER CONNECTION - GW...    2730.00   
2                          CARNE CHARQUE DT PARAISO 400G     122.30   
3                     SALAME ITALIANO FATIADO DALIA 100G      93.48   
4                          CARNE CHARQUE DT PARAISO 400G     122.30   
...                                                  ...        ...   
85830     

In [1]:
import pyodbc
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

# Initialize a list to log queries
query_log = []

def get_informix_connection():
    driver = 'informix_Producao_BCARGA'
    user = 'classprod'
    password = 'yY5eQ4vT'
    database = 'central'
    server = 'bd1hom'
    
    connection_string = f'DSN={driver};UID={user};PWD={password};DATABASE={database};SERVER={server}'
    connection = pyodbc.connect(connection_string)
    return connection

def query_informix(date_range, cnpjs):
    cnpj_str = ', '.join(f"'{cnpj}'" for cnpj in cnpjs)
    query = f"""
    SELECT nfe.sqnfeletronica, 
           hin.nrdocumento AS cnpj, 
           itemnfe.dsproduto, 
           itemnfe.vlproduto, 
           itemnfe.qtproduto, 
           itemnfe.vlbasecalcicms, 
           itemnfe.sgunidmedcom
    FROM tbfis_nfeletronica nfe
    JOIN tbfis_itemnfe itemnfe ON nfe.sqnfeletronica = itemnfe.sqnfeletronica
    JOIN tbfis_hinfeletron hin ON nfe.sqemitente = hin.sqhinfeletron
    WHERE hin.nrdocumento IN ({cnpj_str})
      AND nfe.dhemissao BETWEEN '{date_range[0]}' AND '{date_range[1]}'
    """
    
    # Log the query
    query_log.append(query)
    
    connection = get_informix_connection()
    
    try:
        result_df = pd.read_sql(query, connection)
        return result_df
    except pyodbc.Error as e:
        print("Error: ", e)
        cursor = connection.cursor()
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        print("Columns in the query: ", columns)
        cursor.close()
    finally:
        connection.close()

def fetch_data_parallel(date_ranges, cnpjs):
    results = []
    with ThreadPoolExecutor(max_workers=40) as executor:
        future_to_date_range = {executor.submit(query_informix, date_range, cnpjs): date_range for date_range in date_ranges}
        for future in as_completed(future_to_date_range):
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
            except Exception as exc:
                print(f"Generated an exception: {exc}")
    
    if results:
        final_df = pd.concat(results, ignore_index=True)
        return final_df
    return pd.DataFrame()

if __name__ == "__main__":
    date_ranges = [
        ('2024-04-10 00:00:00', '2024-04-10 23:59:59')
    ]
    df_contribuintes = pd.read_excel(r'C:\Users\jamil\OneDrive\Área de Trabalho\Infomix-Sefaz\arquivos\CNPJS_07_06_24.xlsx')
    cnpj_contribuintes = df_contribuintes['CNPJ'].astype(str).tolist()
    cnpj_contribuintes = [re.sub(r'\D', '', cnpj).zfill(14) for cnpj in cnpj_contribuintes]
    lote = cnpj_contribuintes
    result_df = fetch_data_parallel(date_ranges, lote)
    
    if not result_df.empty:
        print(result_df)
        
    # Write the log to a text file
    with open("query_log.txt", "w") as log_file:
        for query in query_log:
            log_file.write(query + "\n")


c:\Users\jamil\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


       sqnfeletronica            cnpj  \
0           598301298  00289351000164   
1           598301298  00289351000164   
2           598320375  00388857000120   
3           598320375  00388857000120   
4           598363453  00388857000120   
...               ...             ...   
85830       598378615  93209765033120   
85831       598378615  93209765033120   
85832       598378615  93209765033120   
85833       598378612  93209765033120   
85834       598317062  93209765033120   

                                               dsproduto  vlproduto  \
0      COND SPLIT GREE G-TOP INVERTER CONNECTION - GW...    5070.00   
1      EVAP SPLIT GREE G-TOP INVERTER CONNECTION - GW...    2730.00   
2                          CARNE CHARQUE DT PARAISO 400G     122.30   
3                     SALAME ITALIANO FATIADO DALIA 100G      93.48   
4                          CARNE CHARQUE DT PARAISO 400G     122.30   
...                                                  ...        ...   
85830     

In [8]:
result_df

,sqnfeletronica,cnpj,dsproduto,vlproduto,qtproduto,vlbasecalcicms,sgunidmedcom
0,598181416,75315333007464,HF.PIMENTAO VERDE,402.50,35.000,0.00,KG9
1,598181558,75315333007464,HF.BATATA LAVADA,1887.60,363.000,0.00,KG9
2,598181560,75315333007464,HF.CENOURA,264.00,32.000,0.00,KG9
3,598181617,75315333007464,HF.REPOLHO BCO,86.40,16.000,0.00,KG9
4,598181989,75315333007464,HF.INHAME DA COSTA,90.00,10.000,0.00,KG9
...,...,...,...,...,...,...,...
59762,598833169,75315333014673,RF.QUEIJO PRATO REGINA LANCHE,7.20,0.147,7.20,KG9
59763,598833169,75315333014673,RF.QUEIJO PROVOLONE REGINA FRAC,30.63,0.454,30.63,KG9
59764,598833169,75315333014673,RF.QUEIJO REINO REGINA FRAC,2.12,0.028,2.12,KG9
59765,599587451,75315333014673,CAMERA CORPORAL,4596.00,2.000,0.00,UND9


In [7]:
import pyodbc
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

def get_informix_connection():
    driver = 'informix_Producao_BCARGA' 
    user = 'classprod'
    password = 'yY5eQ4vT'
    database = 'central'
    server = 'bd1hom'
    
    connection_string = f'DSN={driver};UID={user};PWD={password};DATABASE={database};SERVER={server}'
    connection = pyodbc.connect(connection_string)
    return connection

def query_informix(date_range, cnpjs):
    cnpj_str = ', '.join(f"'{cnpj}'" for cnpj in cnpjs)
    query = f"""
    SELECT nfe.sqnfeletronica, 
           hin.nrdocumento AS cnpj, 
           itemnfe.dsproduto, 
           itemnfe.vlproduto, 
           itemnfe.qtproduto, 
           itemnfe.vlbasecalcicms, 
           itemnfe.sgunidmedcom
    FROM tbfis_nfeletronica nfe
    JOIN tbfis_itemnfe itemnfe ON nfe.sqnfeletronica = itemnfe.sqnfeletronica
    JOIN tbfis_hinfeletron hin ON nfe.sqemitente = hin.sqhinfeletron
    WHERE hin.nrdocumento IN ('75315333007464', '75315333008940', '75315333011496', '75315333014673')
      AND nfe.dhemissao BETWEEN '{date_range[0]}' AND '{date_range[1]}'
    """
    connection = get_informix_connection()
    
    try:
        result_df = pd.read_sql(query, connection)
        return result_df
    except pyodbc.Error as e:
        print("Error: ", e)
        cursor = connection.cursor()
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        print("Columns in the query: ", columns)
        cursor.close()
    finally:
        connection.close()

def fetch_data_parallel(date_ranges, cnpjs):
    results = []
    with ThreadPoolExecutor(max_workers=40) as executor:
        future_to_date_range = {executor.submit(query_informix, date_range, cnpjs): date_range for date_range in date_ranges}
        for future in as_completed(future_to_date_range):
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
            except Exception as exc:
                print(f"Generated an exception: {exc}")
    
    if results:
        final_df = pd.concat(results, ignore_index=True)
        return final_df
    return pd.DataFrame()

if __name__ == "__main__":
    date_ranges = [
        ('2024-04-10 00:00:00', '2024-04-19 23:59:59')
    ]
    df_contribuintes = pd.read_excel(r'C:\Users\jamil\OneDrive\Área de Trabalho\Infomix-Sefaz\arquivos\CNPJS_07_06_24.xlsx')
    cnpj_contribuintes = df_contribuintes['CNPJ'].astype(str).tolist()
    cnpj_contribuintes = [re.sub(r'\D', '', cnpj).zfill(14) for cnpj in cnpj_contribuintes]
    lote = cnpj_contribuintes
    result_df = fetch_data_parallel(date_ranges, lote)
    if not result_df.empty:
        print(result_df)


c:\Users\jamil\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


       sqnfeletronica            cnpj                        dsproduto  \
0           598181416  75315333007464                HF.PIMENTAO VERDE   
1           598181558  75315333007464                 HF.BATATA LAVADA   
2           598181560  75315333007464                       HF.CENOURA   
3           598181617  75315333007464                   HF.REPOLHO BCO   
4           598181989  75315333007464               HF.INHAME DA COSTA   
...               ...             ...                              ...   
59762       598833169  75315333014673    RF.QUEIJO PRATO REGINA LANCHE   
59763       598833169  75315333014673  RF.QUEIJO PROVOLONE REGINA FRAC   
59764       598833169  75315333014673      RF.QUEIJO REINO REGINA FRAC   
59765       599587451  75315333014673                  CAMERA CORPORAL   
59766       599757121  75315333014673                           EDA50K   

       vlproduto  qtproduto  vlbasecalcicms sgunidmedcom  
0         402.50     35.000            0.00         

## **Extract - Parquet**

In [2]:
import pyodbc
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import re

query_log = []

def get_informix_connection():
    driver = 'informix_Producao_BCARGA'
    user = 'classprod'
    password = 'yY5eQ4vT'
    database = 'central'
    server = 'bd1hom'
    
    connection_string = f'DSN={driver};UID={user};PWD={password};DATABASE={database};SERVER={server}'
    connection = pyodbc.connect(connection_string)
    return connection

def query_informix(date_range, cnpjs):
    cnpj_str = ', '.join(f"'{cnpj}'" for cnpj in cnpjs)
    query = f"""
    SELECT nfe.sqnfeletronica, 
           hin.nrdocumento AS cnpj, 
           itemnfe.dsproduto, 
           itemnfe.vlproduto, 
           itemnfe.qtproduto, 
           itemnfe.vlbasecalcicms, 
           itemnfe.sgunidmedcom
    FROM tbfis_nfeletronica nfe
    JOIN tbfis_itemnfe itemnfe ON nfe.sqnfeletronica = itemnfe.sqnfeletronica
    JOIN tbfis_hinfeletron hin ON nfe.sqemitente = hin.sqhinfeletron
    WHERE hin.nrdocumento IN ({cnpj_str})
      AND nfe.dhemissao BETWEEN '{date_range[0]}' AND '{date_range[1]}'
    """
    
    query_log.append(query)
    
    connection = get_informix_connection()
    
    try:
        result_df = pd.read_sql(query, connection)
        return result_df
    except pyodbc.Error as e:
        print("Error: ", e)
        cursor = connection.cursor()
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        print("Columns in the query: ", columns)
        cursor.close()
    finally:
        connection.close()

def fetch_data_parallel(date_ranges, cnpjs):
    results = []
    with ThreadPoolExecutor(max_workers=40) as executor:
        future_to_date_range = {executor.submit(query_informix, date_range, cnpjs): date_range for date_range in date_ranges}
        for future in as_completed(future_to_date_range):
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
            except Exception as exc:
                print(f"Generated an exception: {exc}")
    
    if results:
        final_df = pd.concat(results, ignore_index=True)
        return final_df
    return pd.DataFrame()

if __name__ == "__main__":
    date_ranges = [
        ('2024-04-10 00:00:00', '2024-04-10 23:59:59')
    ]
    df_contribuintes = pd.read_excel(r'C:\Users\jamil\OneDrive\Área de Trabalho\Infomix-Sefaz\arquivos\CNPJS_07_06_24.xlsx')
    cnpj_contribuintes = df_contribuintes['CNPJ'].astype(str).tolist()
    cnpj_contribuintes = [re.sub(r'\D', '', cnpj).zfill(14) for cnpj in cnpj_contribuintes]
    lote = cnpj_contribuintes[:20]
    result_df = fetch_data_parallel(date_ranges, lote)
    
    if not result_df.empty:
        print(result_df)
        
        result_df.to_parquet('result_output.parquet', engine='pyarrow', index=False)
    
    with open("query_log.txt", "w") as log_file:
        for query in query_log:
            log_file.write(query + "\n")


c:\Users\jamil\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


     sqnfeletronica            cnpj  \
0         598301298  00289351000164   
1         598301298  00289351000164   
2         598320375  00388857000120   
3         598320375  00388857000120   
4         598363453  00388857000120   
..              ...             ...   
824       598360239  01115454000332   
825       598360242  01115454000332   
826       598360242  01115454000332   
827       598360242  01115454000332   
828       598360242  01115454000332   

                                             dsproduto  vlproduto  qtproduto  \
0    COND SPLIT GREE G-TOP INVERTER CONNECTION - GW...    5070.00        1.0   
1    EVAP SPLIT GREE G-TOP INVERTER CONNECTION - GW...    2730.00        1.0   
2                        CARNE CHARQUE DT PARAISO 400G     122.30       10.0   
3                   SALAME ITALIANO FATIADO DALIA 100G      93.48       12.0   
4                        CARNE CHARQUE DT PARAISO 400G     122.30       10.0   
..                                                 

In [3]:
result_df

,sqnfeletronica,cnpj,dsproduto,vlproduto,qtproduto,vlbasecalcicms,sgunidmedcom
0,598301298,00289351000164,COND SPLIT GREE G-TOP INVERTER CONNECTION - GW...,5070.00,1.0,5070.00,UN
1,598301298,00289351000164,EVAP SPLIT GREE G-TOP INVERTER CONNECTION - GW...,2730.00,1.0,2730.00,UN
2,598320375,00388857000120,CARNE CHARQUE DT PARAISO 400G,122.30,10.0,122.30,UN
3,598320375,00388857000120,SALAME ITALIANO FATIADO DALIA 100G,93.48,12.0,93.48,UN
4,598363453,00388857000120,CARNE CHARQUE DT PARAISO 400G,122.30,10.0,122.30,UN
...,...,...,...,...,...,...,...
824,598360239,01115454000332,DORE LIMAO 06X2000 ML,34.52,2.0,34.52,PCT
825,598360242,01115454000332,CAJUI 12X250 ML,13.58,1.0,13.58,PCT
826,598360242,01115454000332,MORMAII PET 06X1000 ML,58.09,2.0,58.09,PCT
827,598360242,01115454000332,MORMAII PET 06X2000 ML,77.09,2.0,77.09,PCT
